In [1]:
import sys, os
import shutil
from google.colab import drive
drive.mount('/content/drive')

path_to_file = '/content/drive/My Drive/Colab Notebooks/'
print(path_to_file)
# change current path to the folder containing "file_name"
os.chdir(path_to_file)

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/


In [2]:
!pip install texthero==1.1.0 -q
!pip install demoji
!pip install emoji
import texthero as hero
import emoji
import demoji
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from texthero import stopwords

pd.set_option('display.max_colwidth', 50)

%matplotlib inline



     |████████████████████████████████| 1.5 MB 7.7 MB/s 
     |████████████████████████████████| 235 kB 40.8 MB/s 
     |████████████████████████████████| 749 kB 52.7 MB/s 
     |████████████████████████████████| 42 kB 732 kB/s 
     |████████████████████████████████| 175 kB 5.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=d382b70e813dbe45bef582565f15cf2cc462fb47700a6a1153161253457fd40a
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_csv('./comments_and_tags.csv', sep = ',', index_col = 'Unnamed: 0')
df.head()

,id,sentiment,search_term,tagger,author,body,created_utc,link_id,permalink,score,subreddit,post_name,filename
0,glngetr,0,ladvc3,SUPRATIK,nevabraun,Thanks but you’ve lost me at \n\n„If look at A...,1612214426,t3_ladvc3,/r/wallstreetbets/comments/ladvc3/just_bought_...,21,wallstreetbets,"Just bought 860 of AMC, greetings from Germany...",comments_batch_6.csv
1,glndurd,1,ladvc3,SUPRATIK,Awake_4E,Awesome 😎! Why the moon 🤔 Let’s shoot AMC out ...,1612213448,t3_ladvc3,/r/wallstreetbets/comments/ladvc3/just_bought_...,25,wallstreetbets,"Just bought 860 of AMC, greetings from Germany...",comments_batch_6.csv
2,glndfos,1,ladvc3,SUPRATIK,Cloud9forreal,If you look at AMCs business page you’ll find ...,1612213289,t3_ladvc3,/r/wallstreetbets/comments/ladvc3/just_bought_...,58,wallstreetbets,"Just bought 860 of AMC, greetings from Germany...",comments_batch_6.csv
3,glnd58d,1,ladvc3,SUPRATIK,MacCoy69,"I bought 20 today, i am also a retarded dumb m...",1612213186,t3_ladvc3,/r/wallstreetbets/comments/ladvc3/just_bought_...,13,wallstreetbets,"Just bought 860 of AMC, greetings from Germany...",comments_batch_6.csv
4,glnd2xt,1,ladvc3,SUPRATIK,Menuler,"Ahh, my fellow Retard. I see the more and more...",1612213163,t3_ladvc3,/r/wallstreetbets/comments/ladvc3/just_bought_...,11,wallstreetbets,"Just bought 860 of AMC, greetings from Germany...",comments_batch_6.csv


In [ ]:




def do_preprocessing_stopwords(df_series):
    custom_pipeline = [hero.preprocessing.fillna, #remove NA
                          hero.preprocessing.lowercase, # lowercase
                          hero.preprocessing.remove_urls,
                          hero.preprocessing.remove_digits, # remove numbers
                          ] 
    df_series = df_series.pipe(hero.clean, custom_pipeline)

    df_series = df_series.apply(remove_emoji) # Remove emojis

     # add custom stopwords
    default_stopwords = stopwords.DEFAULT
    new_words = ['already','also','comment','delete','even','literally','lolol','lololol','lolz','lols','lot','loll','lolololol','lolll','mean','na',
                'point','post','probably','put','reddit','remove','see','something','want','well']
    custom_stopwords = default_stopwords.union(set(new_words))
    df_series = hero.preprocessing.remove_stopwords(df_series, custom_stopwords) # remove stopwords
    lemmatize_text = df_series.apply(words_lemmatizer)
    
    df_series = hero.preprocessing.remove_whitespace(df_series) # remove redundant whitespace as the last step
    
    return df_series

In [4]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.7.0
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import os
import pandas as pd
tf.get_logger().setLevel('ERROR')
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.layers import Input, Dense, Embedding, Activation, Flatten, Dropout
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Concatenate, SimpleRNN,Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 462 kB 11.1 MB/s 
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 352 kB 40.7 MB/s 
     |████████████████████████████████| 237 kB 46.5 MB/s 
     |████████████████████████████████| 1.2 MB 56.0 MB/s 
     |████████████████████████████████| 47.8 MB 41 kB/s 
     |████████████████████████████████| 99 kB 4.3 MB/s 
     |████████████████████████████████| 90 kB 7.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.6 MB/s 
     |████████████████████████████████| 43 kB 890 kB/s 
     |████████████████████████████████| 596 kB 36.4 MB/s 


In [5]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Running on TPU  ['10.65.208.226:8470']


In [6]:

!pip install SentencePiece
!pip install transformers

from transformers import BertTokenizer,BertModel, BertConfig, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.7.0
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import os


     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 35.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.4 MB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 


# BERT w preprocessing

In [15]:
## zx modification for bert

def do_preprocessing(df_series):
  def punctuation_series(series):
    def punctuation(text):
      import re
      text = re.sub(r'[^a-zA-Z0-9?!\.]', ' ', text)
      return text
    series = series.apply(punctuation)
    return series
  
  def emoji_series(series):
    import emoji
    series = series.apply(emoji.demojize) # Remove emojis
    return series

    
  custom_pipeline = [hero.preprocessing.fillna, #remove NA
                        hero.preprocessing.lowercase, # lowercase
                        hero.preprocessing.remove_urls, # remove urls
                        hero.preprocessing.remove_digits, # remove numbers
                        emoji_series, # deemoji
                        punctuation_series, #remove anything that doesnt help in Bert, only punctuation kept is .?!
                        hero.preprocessing.remove_whitespace # remove white spaces
                        ] 
  df_series = df_series.pipe(hero.clean, custom_pipeline)
  return df_series

In [16]:
df['clean'] = do_preprocessing(df['body'])

In [ ]:
df['splitout']=df['clean'].apply([lambda word: word.split()])
df['len'] = df['splitout'].apply(lambda x:len(x))

print('No. of data points with more than 100 words')
print(len(df[df['len']>100])/len(df))
print('No. of data points with more than 200 words')
print(len(df[df['len']>200])/len(df))

No. of data points with more than 100 words
0.012026239067055394
No. of data points with more than 200 words
0.0020043731778425656


In [17]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)
def mask_inputs_for_bert(comments,max_len):
  input_ids = []
  attention_masks = []
  for comment in comments:
    encoded_dict = tokenizer.encode_plus(
        comment,
        add_special_tokens = True,
        max_length = max_len,
        pad_to_max_length = True,
        return_attention_mask= True
    )
    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])
  input_ids = tf.convert_to_tensor(input_ids)
  attention_masks = tf.convert_to_tensor(attention_masks)
  return input_ids, attention_masks



In [18]:
df['sentiment_binary'] = df['sentiment'].apply(lambda x: 1 if x == 1 else 0 )
labels = df["sentiment"].tolist()
labels_binary = df["sentiment_binary"].tolist()
data = df['clean']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,labels_binary, test_size=0.25, random_state=42)

max_len = 100
train_inp, train_mask  = mask_inputs_for_bert(X_train,max_len)
test_inp, test_mask = mask_inputs_for_bert(X_test,max_len)
train_label = tf.convert_to_tensor(y_train)
test_label = tf.convert_to_tensor(y_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
df['sentiment_binary'] = df['sentiment'].apply(lambda x: 1 if x == 1 else 0 )
labels_binary = df["sentiment_binary"].tolist()
data = df['clean']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,labels_binary, test_size=0.25, random_state=42)

max_len = 100
train_inp, train_mask  = mask_inputs_for_bert(X_train,max_len)
test_inp, test_mask = mask_inputs_for_bert(X_test,max_len)
train_label = tf.convert_to_tensor(y_train)
test_label = tf.convert_to_tensor(y_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
def build_classifier_model():
  model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tf.metrics.SparseCategoricalAccuracy('accuracy')

  epochs = 30
  steps_per_epoch = 40 
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  return model
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  bert = build_classifier_model()
  
bert.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('bert_project_v2.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True, save_weights_only = True)
bert.fit([train_inp, train_mask],\
                 train_label,\
                 batch_size=100,\
                 epochs = 30,\
                 validation_data = ([test_inp, test_mask],test_label),
                callbacks = [mc])


Epoch 1/30
42/42 [==============================] - ETA: 0s - loss: 0.7099 - accuracy: 0.5078
Epoch 1: val_accuracy improved from -inf to 0.60714, saving model to bert_project_v2.h5
42/42 [==============================] - 174s 2s/step - loss: 0.7099 - accuracy: 0.5078 - val_loss: 0.6662 - val_accuracy: 0.6071
Epoch 2/30
42/42 [==============================] - ETA: 0s - loss: 0.6063 - accuracy: 0.6698
Epoch 2: val_accuracy improved from 0.60714 to 0.71574, saving model to bert_project_v2.h5
42/42 [==============================] - 9s 220ms/step - loss: 0.6063 - accuracy: 0.6698 - val_loss: 0.5490 - val_accuracy: 0.7157
Epoch 3/30
42/42 [==============================] - ETA: 0s - loss: 0.5062 - accuracy: 0.7668
Epoch 3: val_accuracy improved from 0.71574 to 0.76239, saving model to bert_project_v2.h5
42/42 [==============================] - 9s 220ms/step - loss: 0.5062 - accuracy: 0.7668 - val_loss: 0.4965 - val_accuracy: 0.7624
Epoch 4/30
42/42 [==============================] - ETA:

In [21]:
bert.load_weights('bert_project_v2.h5')
df['sentiment_binary'] = df['sentiment'].apply(lambda x: 1 if x == 1 else 0 )
input, mask  = mask_inputs_for_bert(df['clean'],max_len)
target = df['sentiment_binary']
prediction = bert.predict([input, mask ])
prediction_class = prediction[0].argmax(axis=1)


from sklearn.metrics import precision_score
from sklearn.metrics import balanced_accuracy_score
print('Precision score = {}'.format(precision_score(target,prediction_class)))
print('Balanced Accuracy score = {}'.format(balanced_accuracy_score(target,prediction_class)))

Precision score = 0.8293424553401748
Balanced Accuracy score = 0.8418799275516726


In [ ]:
pd.Series(prediction_class).value_counts()

0    2857
1    2631
dtype: int64

In [ ]:
frame = df.reset_index().drop('index',axis=1).join(pd.DataFrame(prediction[0],columns = ['Class_0 Logits BERT', 'CLass_1 Logits BERT']))
frame.to_csv('combined_tags_bert_logits_v1.csv')

In [ ]:
data['clean'] = do_preprocessing(data['body'])

In [ ]:
bert.load_weights('bert_project_v2.h5')

input, mask  = mask_inputs_for_bert(data['clean'],max_len)
prediction = bert.predict([input, mask ])
prediction_class = prediction[0].argmax(axis=1)
pd.Series(prediction_class).value_counts(normalize=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0    0.663597
1    0.336403
dtype: float64

In [ ]:
frame = data.join(pd.DataFrame(prediction[0],columns = ['Class_0 Logits BERT', 'CLass_1 Logits BERT']))
frame.to_csv('combined_comments_bert_logits_v1.csv')

# Bert preprocessing with stop words removal 

In [7]:
## zx modification for bert

def do_preprocessing_v2(df_series):
  def punctuation_series(series):
    def punctuation(text):
      import re
      text = re.sub(r'[^a-zA-Z0-9?!\.]', ' ', text)
      return text
    series = series.apply(punctuation)
    return series
  
  def emoji_series(series):
    import emoji
    series = series.apply(emoji.demojize) # Remove emojis
    return series
  

  #add custom stopwords
  # default_stopwords = stopwords.DEFAULT

    
  custom_pipeline = [hero.preprocessing.fillna, #remove NA
                        hero.preprocessing.lowercase, # lowercase
                        hero.preprocessing.remove_urls, # remove urls
                        hero.preprocessing.remove_digits, # remove numbers
                        emoji_series, # deemoji
                        punctuation_series, #remove anything that doesnt help in Bert, only punctuation kept is .?!
                        hero.preprocessing.remove_whitespace # remove white spaces
                        ] 
  df_series = df_series.pipe(hero.clean, custom_pipeline)

  custom_stopwords = ['amc','bb','clov','pltr','gme','nok','meta','fb','amzn','wish','tsla']
  # custom_stopwords = default_stopwords.union(set(new_words))
  df_series = hero.preprocessing.remove_stopwords(df_series, custom_stopwords) # remove stopwords
  return df_series

In [8]:
df['clean2'] = do_preprocessing_v2(df['body'])

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)
def mask_inputs_for_bert(comments,max_len):
  input_ids = []
  attention_masks = []
  for comment in comments:
    encoded_dict = tokenizer.encode_plus(
        comment,
        add_special_tokens = True,
        max_length = max_len,
        pad_to_max_length = True,
        return_attention_mask= True
    )
    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])
  input_ids = tf.convert_to_tensor(input_ids)
  attention_masks = tf.convert_to_tensor(attention_masks)
  return input_ids, attention_masks



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
df['sentiment_binary'] = df['sentiment'].apply(lambda x: 1 if x == 1 else 0 )
labels_binary = df["sentiment_binary"].tolist()
data = df['clean2']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,labels_binary, test_size=0.25, random_state=42)

max_len = 100
train_inp, train_mask  = mask_inputs_for_bert(X_train,max_len)
test_inp, test_mask = mask_inputs_for_bert(X_test,max_len)
train_label = tf.convert_to_tensor(y_train)
test_label = tf.convert_to_tensor(y_test)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
def build_classifier_model():
  model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tf.metrics.SparseCategoricalAccuracy('accuracy')

  epochs = 30
  steps_per_epoch = 40 
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  return model
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  bert = build_classifier_model()
  
bert.summary()

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('bert_project_v3.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True, save_weights_only = True)
bert.fit([train_inp, train_mask],\
                 train_label,\
                 batch_size=100,\
                 epochs = 30,\
                 validation_data = ([test_inp, test_mask],test_label),
                callbacks = [mc])


Epoch 1/30
42/42 [==============================] - ETA: 0s - loss: 0.6983 - accuracy: 0.5323
Epoch 1: val_accuracy improved from -inf to 0.64431, saving model to bert_project_v3.h5
42/42 [==============================] - 167s 2s/step - loss: 0.6983 - accuracy: 0.5323 - val_loss: 0.6522 - val_accuracy: 0.6443
Epoch 2/30
42/42 [==============================] - ETA: 0s - loss: 0.6021 - accuracy: 0.6871
Epoch 2: val_accuracy improved from 0.64431 to 0.72012, saving model to bert_project_v3.h5
42/42 [==============================] - 9s 226ms/step - loss: 0.6021 - accuracy: 0.6871 - val_loss: 0.5640 - val_accuracy: 0.7201
Epoch 3/30
42/42 [==============================] - ETA: 0s - loss: 0.5064 - accuracy: 0.7619
Epoch 3: val_accuracy did not improve from 0.72012
42/42 [==============================] - 6s 148ms/step - loss: 0.5064 - accuracy: 0.7619 - val_loss: 0.5700 - val_accuracy: 0.7172
Epoch 4/30
42/42 [==============================] - ETA: 0s - loss: 0.3887 - accuracy: 0.8392
Ep

In [ ]:
input, mask  = mask_inputs_for_bert(df['clean2'],max_len)
target = df['sentiment_binary']
prediction = bert.predict([input, mask ])
prediction_class = prediction[0].argmax(axis=1)


from sklearn.metrics import precision_score
from sklearn.metrics import balanced_accuracy_score
print('Precision score = {}'.format(precision_score(target,prediction_class)))
print('Balanced Accuracy score = {}'.format(balanced_accuracy_score(target,prediction_class)))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Precision score = 0.9298312883435583
Balanced Accuracy score = 0.9298312883435583


In [14]:
bert.load_weights('bert_project_v3.h5')
input, mask  = mask_inputs_for_bert(df['clean2'],max_len)
target = df['sentiment_binary']
prediction = bert.predict([input, mask ])
prediction_class = prediction[0].argmax(axis=1)


from sklearn.metrics import precision_score
from sklearn.metrics import balanced_accuracy_score
print('Precision score = {}'.format(precision_score(target,prediction_class)))
print('Balanced Accuracy score = {}'.format(balanced_accuracy_score(target,prediction_class)))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Precision score = 0.89223337049424
Balanced Accuracy score = 0.9115230129980822


In [ ]:
pd.Series(prediction_class).value_counts()

0    2797
1    2691
dtype: int64

In [ ]:
frame = df.reset_index().drop('index',axis=1).join(pd.DataFrame(prediction[0],columns = ['Class_0 Logits BERT', 'CLass_1 Logits BERT']))
frame.to_csv('combined_tags_bert_logits_v2.csv')

## Finally generating on full data

In [ ]:
data = pd.read_csv('combined_comments.csv')
data = data[data['body']!='[removed]']
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,filename,author,body,created_utc,id,link_id,permalink,score,subreddit,search_term,post_name
1,SUPRATIK_0.csv,okgeezeok,https://youtu.be/gmq1ueWGKgY,1612139703.0,gljgye5,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...
2,SUPRATIK_0.csv,LegitimateInjury5720,And to da moon,1611968923.0,glbe5gt,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...
4,SUPRATIK_0.csv,jawnlerdoe,I’m all in at open too. We’re in this together.,1611895308.0,gl71p20,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...
7,SUPRATIK_0.csv,JokersKnight,Invest it has such a strong really behind it,1611891585.0,gl6tpoi,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...
9,SUPRATIK_0.csv,JoeWelburg,AMC will legit go through the roof tomorrow,1611889728.0,gl6pihh,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,2.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...
...,...,...,...,...,...,...,...,...,...,...,...
1092975,ZONGXIAN_2.csv,MyNi_NotYourNi,... and all they got was a weird haircut.,1639102296,hnxm4z1,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,27,wallstreetbets,rcx695,12/9 TSLA Split They Said
1092976,ZONGXIAN_2.csv,Ok_Bottle_2198,Musk fanboys are definitely the dumbest breed ...,1639102292,hnxm4nh,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,26,wallstreetbets,rcx695,12/9 TSLA Split They Said
1092977,ZONGXIAN_2.csv,inntw-inutw,"Alright, ima head out .. to get my knee pads",1639102206,hnxlxye,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,7,wallstreetbets,rcx695,12/9 TSLA Split They Said
1092978,ZONGXIAN_2.csv,Forrest_GUHmp,Where's the loss porn? Did that guy with stupi...,1639101702,hnxkt8u,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,14,wallstreetbets,rcx695,12/9 TSLA Split They Said


In [ ]:
data['clean2'] = do_preprocessing_v2(data['body'])

In [ ]:
bert.load_weights('bert_project_v3.h5')
max_len=100
input, mask  = mask_inputs_for_bert(data['clean2'],max_len)
prediction = bert.predict([input, mask ])
prediction_class = prediction[0].argmax(axis=1)
pd.Series(prediction_class).value_counts()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0    499657
1    310532
dtype: int64

In [ ]:
data.reset_index().drop('index',axis=1)

,filename,author,body,created_utc,id,link_id,permalink,score,subreddit,search_term,post_name,clean2
0,SUPRATIK_0.csv,okgeezeok,https://youtu.be/gmq1ueWGKgY,1612139703.0,gljgye5,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...,
1,SUPRATIK_0.csv,LegitimateInjury5720,And to da moon,1611968923.0,glbe5gt,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...,and to da moon
2,SUPRATIK_0.csv,jawnlerdoe,I’m all in at open too. We’re in this together.,1611895308.0,gl71p20,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...,i m all in at open too. we re in this together.
3,SUPRATIK_0.csv,JokersKnight,Invest it has such a strong really behind it,1611891585.0,gl6tpoi,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,1.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...,invest it has such a strong really behind it
4,SUPRATIK_0.csv,JoeWelburg,AMC will legit go through the roof tomorrow,1611889728.0,gl6pihh,t3_l7c2a3,/r/wallstreetbets/comments/l7c2a3/fuck_the_hed...,2.0,wallstreetbets,l7c2a3,Fuck the hedge funds; diamond hands. AMC to th...,will legit go through the roof tomorrow
...,...,...,...,...,...,...,...,...,...,...,...,...
810184,ZONGXIAN_2.csv,MyNi_NotYourNi,... and all they got was a weird haircut.,1639102296,hnxm4z1,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,27,wallstreetbets,rcx695,12/9 TSLA Split They Said,... and all they got was a weird haircut.
810185,ZONGXIAN_2.csv,Ok_Bottle_2198,Musk fanboys are definitely the dumbest breed ...,1639102292,hnxm4nh,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,26,wallstreetbets,rcx695,12/9 TSLA Split They Said,musk fanboys are definitely the dumbest breed ...
810186,ZONGXIAN_2.csv,inntw-inutw,"Alright, ima head out .. to get my knee pads",1639102206,hnxlxye,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,7,wallstreetbets,rcx695,12/9 TSLA Split They Said,alright ima head out .. to get my knee pads
810187,ZONGXIAN_2.csv,Forrest_GUHmp,Where's the loss porn? Did that guy with stupi...,1639101702,hnxkt8u,t3_rcx695,/r/wallstreetbets/comments/rcx695/129_tsla_spl...,14,wallstreetbets,rcx695,12/9 TSLA Split They Said,where s the loss porn? did that guy with stupi...


In [ ]:
frame = data.reset_index().drop('index',axis=1).join(pd.DataFrame(prediction[0],columns = ['Class_0 Logits BERT', 'CLass_1 Logits BERT']))
frame.to_csv('combined_comments_bert_logits_v2.csv')

In [ ]:
df['sentiment_binary'].value_counts(normalize=True)

0    0.526239
1    0.473761
Name: sentiment_binary, dtype: float64

In [ ]:
pd.Series(prediction_class).value_counts(normalize=True)

0    0.616717
1    0.383283
dtype: float64